In [ ]:
#| default_exp ml_model

In [ ]:
#| export
from relax.import_essentials import *
from relax.data_module import DataModule, DEFAULT_DATA_CONFIGS
from relax.utils import validate_configs
from relax.base import *
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from urllib.request import urlretrieve

Using JAX backend.


## ML Module

In [ ]:
#| export
class MLPBlock(keras.layers.Layer):
    """MLP block with leaky relu activation and dropout."""

    def __init__(
        self, 
        output_size: int, 
        dropout_rate: float = 0.3,
    ):
        super().__init__()
        self.output_size = output_size
        self.dropout_rate = dropout_rate

    def build(self, input_shape):
        self.dense = keras.layers.Dense(
            self.output_size, activation='leaky_relu')
        self.dropout = keras.layers.Dropout(self.dropout_rate)

    def call(self, x, training=False):
        x = self.dense(x)
        x = self.dropout(x, training=training)
        return x

@keras.saving.register_keras_serializable()
class MLP(keras.Model):
    """MLP model with multiple MLP blocks and a dense layer at the end."""
    
    def __init__(
        self, 
        sizes: list, 
        output_size: int = 2,
        dropout_rate: float = 0.3,
        **kwargs
    ):
        super().__init__(**kwargs)
        self.blocks = []
        for size in sizes:
            self.blocks.append(MLPBlock(size, dropout_rate))
        self.dense = keras.layers.Dense(output_size, activation='softmax')

    def call(self, x, training=False):
        for block in self.blocks:
            x = block(x, training=training)
        return self.dense(x)

    def get_config(self):
        return {
            'sizes': [block.output_size for block in self.blocks],
            'output_size': self.dense.units,
            'dropout_rate': self.blocks[0].dropout_rate,
        }

In [ ]:
#| export
class MLModuleConfig(BaseParser):
    """Configurator of `MLModule`."""
    
    sizes: List[int] = Field(description="List of hidden layer sizes.")
    output_size: int = Field(2, description="The number of output classes.")
    dropout_rate: float = Field(0.3, description="Dropout rate.")
    lr: float = Field(1e-3, description="Learning rate.")
    opt_name: str = Field("adam", description="Optimizer name.")
    loss: str = Field("sparse_categorical_crossentropy", description="Loss function name.")
    metrics: List[str] = Field(["accuracy"], description="List of metrics names.")


In [ ]:
#| export
class MLModule(BaseModule, TrainableMixedin, PredFnMixedin):
    def __init__(self, config: MLModuleConfig, *, model: keras.Model = None, name: str = None):
        config = validate_configs(config, MLModuleConfig)
        self.model = self._init_model(config, model)
        self._is_trained = False
        super().__init__(config, name=name)

    def _init_model(self, config: MLModuleConfig, model: keras.Model):
        if model is None:
            model = MLP(
                sizes=config.sizes,
                output_size=config.output_size,
                dropout_rate=config.dropout_rate
            )
            model.compile(
                optimizer=keras.optimizers.get({
                    'class_name': config.opt_name, 
                    'config': {'learning_rate': config.lr}
                }),
                loss=config.loss,
                metrics=config.metrics
            )
        return model
            

    def train(
        self, 
        data: DataModule, 
        batch_size: int = 128,
        epochs: int = 10,
        **fit_kwargs
    ):
        if isinstance(data, DataModule):
            X_train, y_train = data['train']
        else:
            X_train, y_train = data
        self.model.fit(
            X_train, y_train, 
            batch_size=batch_size, 
            epochs=epochs,
            **fit_kwargs
        )
        self._is_trained = True
        return self.model
    
    @property
    def is_trained(self) -> bool:
        return self._is_trained
    
    def save(self, path):
        path = Path(path)
        if not path.exists():
            path.mkdir(parents=True)
        # self.model.save_weights(path / "model.weights.h5", overwrite=True)
        self.model.save(path / "model.keras")
        with open(path / "config.json", "w") as f:
            json.dump(self.config.dict(), f)

    @classmethod
    def load_from_path(cls, path):
        path = Path(path)
        config = MLModuleConfig(**json.load(open(path / "config.json")))
        model = keras.models.load_model(path / "model.keras")
        module = cls(config, model=model)
        module._is_trained = True
        return module
    
    def pred_fn(self, x):
        if not self.is_trained:
            raise ValueError("Model is not trained.")
        return self.model(x)

In [ ]:
X, y = make_classification(
    n_samples=5000, n_features=10, n_informative=5, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
model = MLModule(
    MLModuleConfig(sizes=[64, 32, 16],)
)
model.train((X_train, y_train), epochs=5)
assert model.is_trained

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


Epoch 1/5
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - accuracy: 0.5581 - loss: 0.7279    
Epoch 2/5
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7460 - loss: 0.5062        
Epoch 3/5
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7960 - loss: 0.4470        
Epoch 4/5
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8195 - loss: 0.4112      
Epoch 5/5
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8251 - loss: 0.3935        


In [ ]:
model.save('tmp/model')

2023-07-23 18:18:25.386186: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
model_1 = MLModule.load_from_path('tmp/model')
assert model_1.is_trained
assert np.allclose(model_1.pred_fn(X_test), model.pred_fn(X_test))

In [ ]:
#| hide
# remove tmp directory
shutil.rmtree('tmp')

## Load ML Module

TODO: Need test cases

In [ ]:
#| export
def download_ml_module(name: str, path: str = None):
    if path is None:
        path = Path('relax_assets') / name / 'model'
    else:
        path = Path(path)
    if not path.exists():
        path.mkdir(parents=True)
    _model_path = f"assets/{name}/model"
    model_url = f"https://github.com/BirkhoffG/ReLax/raw/master/{_model_path}/model.keras"
    config_url = f"https://github.com/BirkhoffG/ReLax/raw/master/{_model_path}/config.json"

    if not (path / "model.keras").exists():
        urlretrieve(model_url, filename=str(path / "model.keras"))
    if not (path / "config.json").exists():
        urlretrieve(config_url, filename=str(path / "config.json"))   
    

def load_ml_module(name: str) -> MLModule:
    """Load the ML module"""

    if name not in DEFAULT_DATA_CONFIGS.keys():
        raise ValueError(f'`data_name` must be one of {DEFAULT_DATA_CONFIGS.keys()}, '
            f'but got data_name={name}.')

    download_ml_module(name)
    return MLModule.load_from_path(f"relax_assets/{name}/model")